# Lade Daten per Twitter API
*Es werden Timeline Daten für jeden aktuellen Bundestags Politiker geladen, der auf Twitter aktiv ist. 
Die Daten werden anschließend in MongoDB abgelegt, wobei jeder Tweet einem Dokument entspricht. Die Namen der User wurden von den Fraktionsseiten gecrawled.*

### API Verbindung aufsetzen

In [2]:
# Tweepy statt twitter Paket
# https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/

import os
import tweepy as tw
import pandas as pd

# Die folgenden Zugangsaten können bei Twitter Developer abgerufen werden, sobald ein Account erstellt wurde
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token_key = os.getenv('access_token_key')
access_token_secret = os.getenv('access_token_secret')

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
print('consumer_key ='+os.getenv('consumer_key'))
print('consumer_secret ='+os.getenv('consumer_secret'))
print('access_token_key ='+os.getenv('access_token_key'))
print('access_token_secret ='+os.getenv('access_token_secret'))

consumer_key =
consumer_secret =O6TB4nfapbkriYFFNjKwbZS2W6N8jfcAnxOgZxMALVH3EquFS8
access_token_key =2357227475-oza7eWuMjiHjpXAJIcc7iimWMqUbiTePs9d986G
access_token_secret =Fu6pH66pQCm4zcH2DKoe9RVcHM2yS6sGAT1VRKNn7jV3b


### API Verbindung und speichern in Mongo DB
In der Collection Twitter_mdp wird jeder Tweet mit allen Metainformationen als ein Dokument gespeichert. Es werden Tweets zu allen auf Twitter aktiven Politikern einer Fraktion gesammelt. Die entsprechenden Namen wurden von den Bundestagsfraktionsseiten gecrawled.

Anfang Juni 2020 wurde zur Qualitätsicherung des Datensatzes außerdem eine manuelle Kontrolle durchgeführt. Dabei fiel auf, dass der Großteil von Abgeordneten einen Twitter Account besitzen, dieser aber nicht auf der Fraktionsseite verlinkt wurde. Dies ist der Fall bei den folgenden Politikern:

"BerndBaumannAfD", "AndreasBleckMdB", "PeterBoehringer", "JuergenBraunAfD", "Marcus_Buehl", "Buettner_MdB", "Tino_Chrupalla", "DroeseSiegbert", "Thomas_Ehrhorn", "PeterFelser", "DrFriesenMdB", "GtzFrmming", "AlexGauland", "Paul_Hampel_BT", "M_HarderKuehnel", "R_Hartwig_AfD", "WaldemarHerdt", "Martin_Hess_AfD", "HilseMdb", "Nicole_Hoechst", "HohmannMdb", "DrHollnagel", "Huber_AfD", "Marc_Jongen", "KestnerJens", "StefanKeuterAfD", "NKleinwaechter", "JoernKoenigAfD", "Dr_Rainer_Kraft", "Frank_Magnitz", "LotharMaier1", "JensMaierAfD", "MalsackWinkeman", "mrosek1958", "MdB_Mueller_AfD", "Jan_Nolte_AfD", "Ulrich_Oehme", "Gerold_Otten", "Frank_Pasemann", "Pohl_MdB", "M_Reichardt_AfD", "Renner_AfD", "ulschzi", "Schneider_AfD",
"Uwe_Schulz_AfD", "Th_Seitz_AfD", "Martin_Sichert", "DirkSpaniel", "h_weyel", "WolfgangWiehle", "Heiko_Wildberg", "ChrWirthMdB", TobiasMPeterka

Die Tweets aus den Timelines dieser Politiker wurden entsprechend im Juni in den Datensatz aufgenommen.

Für die folgenden Namen gab es Auffälligkeiten, weshalb sie nicht enthalten sind (Grund angegeben):
* afdBystron ist ein Sartire Account und wurde vmlt. fälschlicherweise auf der AFD Homepage verlinkt
-> Am 10.05. wurde er durch den echten Account ersetzt: PetrBystronAfD 
* EspendillerM hat Stand 10.05. seinen Screen_name zu DrMEspendiller geändert (hat wohl Promoviert)
-> Die beiden Accounts scheinen verknüpft zu sein, Duplikate werden erkannt (auch wenn der screen_name ein neuer ist)
* Albrecht (protected)

In [3]:
import pymongo

client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

db = client['Twitter']
# All_Tweets_collection = db['twitter_mdp_ex_date_proj'] # Flache Collection mit weniger Feldern
All_Tweets_collection = db['Twitter_mdp_extend']         # Basis in der zunächst alle Tweets gesammelt werden

Über den Zeitraum Feb-Aug 2020 wurde der Datensatz zu verschiedenen Zeitpunkten aktualisiert. Dabei ergaben sich folgende Probleme:
* die Option "since" in $\texttt{.user_timeline}$ wird ignoriert wenn man gleichtzeitig "count" oder "page" verwendet
* beim laden in die modifizierte Collection Twitter_mp_datetime wurde ein duplicate error geworfen
* durch die Arbeit auf zwei Computern muss die Datenbank bei Aktualisierung synchronisiert werden

In [ ]:
import json

with open('Namen_AFD.json') as json_file:
    AFD = json.load(json_file)
    
# AFD = AFD[33:35]
#print(AFD)

for Abgeordneter in AFD:
    i = 1
    user = Abgeordneter
    print(user)
    all_tweets = []
    while i <= 100 :
        tweets_of_page = api.user_timeline(screen_name = user, tweet_mode = "extended", count = 200, page = i, include_rts = True)
        #print(len(tweets_of_page))
        all_tweets = all_tweets + tweets_of_page
        i=i+1
        if len(tweets_of_page) == 0:
            break    # no more tweets from this user
            
    for tweet in all_tweets:
        All_Tweets_collection.insert_one(tweet._json)
    print(len(all_tweets))
# DroeseSiegbert -> Name geändert (alter Account gesperrt)

StBrandner
3207
SteffenKotre
696
Witt_Uwe
1116
EnricoKomning
416
MdB_Lucassen
263
Beatrix_vStorch
3199
Rene_Springer
2861
Alice_Weidel
2772
DrRobbySchlund
29
Paul_Podolay
54
AfDProtschka
3178
ttte94
61
S_Muenzenmaier
1663
CorinnaMiazga
300
Jacobi_AfD
709
Leif_Erik_Holm
1524
UdoHemmelgarn
3109
Jochen_Haug
739
KayGottschalk1
1461
axelgehrke
138
GtzFrmming
3189
Frohnmaier_AfD
1185
DrMEspendiller
1660
ElsnervonGronow
3086
GottfriedCurio
581
JoanaCotar
3210
BerndBaumannAfD
99
AndreasBleckMdB
906
PeterBoehringer
567
JuergenBraunAfD
3207
Marcus_Buehl
611
Buettner_MdB
561
Tino_Chrupalla
827
Siegbert_Droese
18
Thomas_Ehrhorn
804
PeterFelser
387
DrFriesenMdB
1291
GtzFrmming
3189
AlexGauland
6
Paul_Hampel_BT
4
M_HarderKuehnel
417
R_Hartwig_AfD
1076
WaldemarHerdt
263
Martin_Hess_AfD
1833
HilseMdb
17
Nicole_Hoechst
2648
HohmannMdb
168
DrHollnagel
196
Huber_AfD
25
Marc_Jongen
353
KestnerJens
801
StefanKeuterAfD
2155
NKleinwaechter
663
JoernKoenigAfD
446
Dr_Rainer_Kraft
865
Frank_Magnitz
810
LotharMa

In [9]:
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/AFD_twitter_clean.json') as json_file:
with open('Namen_AFD.json') as json_file:
    AFD = json.load(json_file)
    
AFD = AFD[33]
print(AFD)

Siegbert_Droese


#### Liste aller AFD Politiker ohne Daten in der Datenbank

Für manche Politiker findet man keine Daten in Twitter_mdp, wenn man mit dem Screen_name aus der Liste AFD_twitter_clean sucht.

Grund: Die Usernamen in der Liste von der Fraktionsseite (_twitter_clean) weichen ab vom tatsächlichen Screen_name

-> Twitter API ist NICHT case sensitiv, MongoDB und Python aber schon!

=> Option mit RegEx zu suchen (_twitter_Namen) ODER einen Tweet laden und exakten user_screen_name speichern (siehe unten - _twitter_realNames)

In [2]:
# import json
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/AFD_twitter_clean.json') as json_file:
#     AFD = json.load(json_file)

# AFD_Name = AFD
# mdp_not_mongodb = []

# for name, n in zip(AFD, range(0,len(AFD))):
#     count = 0
#     #cursor = All_Tweets_collection.find({"user.screen_name": {'$regex':name,'$options':'i'}})
#     cursor = All_Tweets_collection.find({"user_screen_name": name})
#     for doc in cursor:
#         count=count+1
        
#     if count == 0:
#         #print(count)
#         print(name)
#         #mdp_not_mongodb.append(name)
#         AFD_Name[n] = All_Tweets_collection.find_one({"user_screen_name": {'$regex':name,'$options':'i'}})['user_screen_name']
   
        
# print(mdp_not_mongodb)
# with open("/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/AFD_twitter_Names.json", "a") as write_file:
# #with open("mdp_not_mongodb_AFD.json", "w") as write_file:
#      json.dump(AFD_Name, write_file)


#### Erstellen der Liste mit den tatsächlichen screen_names 

In [1]:
# import json

# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/AFD_twitter_clean.json') as json_file:
#     AFD= json.load(json_file)
    
# #AFD = AFD[]
# names = []

# for Abgeordneter in AFD:
#     user = Abgeordneter
#     print(user)
#     all_tweets = api.user_timeline(screen_name = user, count = 1, include_rts = True)
#     for tweet in all_tweets:
#         name = tweet.user.screen_name
#         names.append(name)
        
# print(names)

# with open ("AFD_twitter_realNames", "w") as write_file:
#     json.dump(names, write_file)
    